<a href="https://colab.research.google.com/github/VGokulPillai/compute_webpage_url_score/blob/main/compute_webpage_url_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy==2.3.7
!pip install textacy
!pip install neomodel

In [ ]:
import spacy
print(spacy.__version__)

2.3.7


In [ ]:
!python -m spacy download en

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
!git clone https://github.com/huggingface/neuralcoref.git

In [ ]:
import os
os.chdir("/content/neuralcoref")
!pip install -r requirements.txt
!pip install -e .

In [ ]:
import logging;
logging.basicConfig(level=logging.INFO)

In [ ]:
import neuralcoref

INFO:neuralcoref:Loading model from /root/.neuralcoref_cache/neuralcoref


In [ ]:
inquiries = { 'name' : "Organic Shelled Hemp seeds",
              'supplier_countries' : "GB",
              'description' :"Hi ,Im looking for Organic shelled hemp seeds -300kgs delivered to Rochdale. The product will have to be handled in a nut free site. We would like a declaration from the supplier to confirm the allergen information."
}

In [ ]:
text = inquiries['description']

In [ ]:
import spacy
from urllib.parse import urlparse


nlp = spacy.load("en_core_web_sm")
print(text)
doc = nlp(text)
# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])


# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)
neuralcoref.add_to_pipe(nlp)
coref = doc._.has_coref
if coref:
  print(doc._.coref_clusters)
  text = nlp(text._.coref_resolved)

In [ ]:
import spacy
import textacy
import knowledgeExtraction
import pandas as pd
import entityRecognitionLinking
import re
from urllib.parse import urlparse
import nltk
from nltk.stem.porter import *
stemmer = PorterStemmer()

from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

nlp = spacy.load("en_core_web_sm")


#Removing Unwanted things
inputText = re.sub(r'\.(?=[^ \W\d])','.,', text)
entityRecognitionLinkingObj = entityRecognitionLinking.EntityRecognitionLinking()
entityRelJson = entityRecognitionLinkingObj.entityRecogLink(text)

stemmer_list = []
for resource in entityRelJson['Resources']:
  print(resource["@surfaceForm"] + ' --> ' + stemmer.stem(resource["@surfaceForm"]))
  stemmer_list.append(stemmer.stem(resource["@surfaceForm"]))
stemmer_list_inquiry_kws = set(stemmer_list)

def compute_webpage_url_score(page_url: str):
  stemmer_list = []
  last_url_path_part = urlparse(page_url).path.strip('/').split('/')[-1]
  if last_url_path_part.endswith('.html'):
    last_url_path_part = last_url_path_part[:-5]
  last_url_path_part = last_url_path_part.split("-")
  for tok in last_url_path_part:
     print(tok + ' --> ' + stemmer.stem(tok))
     stemmer_list.append(stemmer.stem(tok))
  product_words_extracted =' '.join(stemmer_list)
  return  product_words_extracted


page_url = "https://impactfoods.co.uk/products/hemp-seeds-shelled"
product_words_extracted = compute_webpage_url_score(page_url)
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('dbpedia_spotlight')
doc = nlp(product_words_extracted)

for ent in doc.ents:
  print(ent.text,ent._.dbpedia_raw_result['@similarityScore'], ent.kb_id_)
  product_words = ent.text

product_words_splited = product_words.split(" ")
product_words_splited = set(product_words_splited)
print("product_words_splited: ",product_words_splited)
print("list_inquiry_keywords: ",stemmer_list_inquiry_kws)
common_words = product_words_splited.intersection(stemmer_list_inquiry_kws)
print("Common_Keywords:",common_words)
webpage_url_score = 2 * len(common_words) / (len(product_words_splited) + len(stemmer_list_inquiry_kws))
print("webpage_url_score:",webpage_url_score)

{
    "@confidence": "0.35",
    "@policy": "whitelist",
    "@sparql": "",
    "@support": "0",
    "@text": "Hi ,Im looking for Organic shelled hemp seeds -300kgs delivered to Rochdale. The product will have to be handled in a nut free site. We would like a declaration from the supplier to confirm the allergen information.",
    "@types": "",
    "Resources": [
        {
            "@URI": "http://dbpedia.org/resource/Hemp",
            "@offset": "35",
            "@percentageOfSecondRank": "0.011092646810311921",
            "@similarityScore": "0.9884877806876949",
            "@support": "2130",
            "@surfaceForm": "hemp",
            "@types": ""
        },
        {
            "@URI": "http://dbpedia.org/resource/Seed",
            "@offset": "40",
            "@percentageOfSecondRank": "0.010469179576643395",
            "@similarityScore": "0.984305762411485",
            "@support": "5889",
            "@surfaceForm": "seeds",
            "@types": ""
        },
  